In [43]:
# conduct simulation specific analyses
import os
from pathlib import Path
import pandas as pd
import numpy as np

CURRENT_DIRECTORY = "C:/STORE/project/scivis2023"
# CURRENT_DIRECTORY = "C:/Users/sonne/Documents/Clark Scholars"

dfPositions = pd.read_csv(str(CURRENT_DIRECTORY)+"/SciVisContest23/viz-calcium/positions/rank_0_positions.txt", header=7, sep=' ')
dropColNames = list(dfPositions.columns[6:12])
dfPositions =  dfPositions.drop(dropColNames, axis=1)
dfPositions.columns = ['local id', 'pos x', 'pos y', 'pos z', 'area', 'type']

regionList = dfPositions['area'].unique().tolist()

def getAvgRegion(ts1, ts2, sim):
    # returns the average number of creations and deletions between two time steps
    directory = str(CURRENT_DIRECTORY) + "/SciVisContest23/viz-" + sim + "/network/changes/"
    resultCreated = pd.DataFrame({'area': regionList, 'avg': [0]*len(regionList)})
    resultDeleted = pd.DataFrame({'area': regionList, 'avg': [0]*len(regionList)})
    resultCreated.set_index('area', inplace=True)
    resultDeleted.set_index('area', inplace=True)
    for file in os.listdir(directory):
        if file.startswith("area_"): # regional data
            data = pd.read_csv(directory+file)
            data['total'] = data['in'] + data['out']
            regionName = file[:-12]
            if file.endswith('created.csv'):
                resultCreated.loc[regionName] = {'avg': data.iloc[ts1//10000:ts2//10000,-1].mean()}
            if file.endswith('deleted.csv'):
                resultDeleted.loc[regionName] = {'avg': data.iloc[ts1//10000:ts2//10000,-1].mean()}
    return (resultCreated, resultDeleted)

def getTimestepRegion(ts, sim):
    # returns the number of creations and deletions at one time step
    directory = str(CURRENT_DIRECTORY) + "/SciVisContest23/viz-" + sim + "/network/changes/"
    resultCreated = pd.DataFrame({'area': regionList, 'created': [0]*len(regionList)})
    resultDeleted = pd.DataFrame({'area': regionList, 'deleted': [0]*len(regionList)})
    resultCreated.set_index('area', inplace=True)
    resultDeleted.set_index('area', inplace=True)
    for file in os.listdir(directory):
        if file.startswith("area_"): # regional data
            data = pd.read_csv(directory+file)
            data['total'] = data['in'] + data['out']
            regionName = file[:-12]
            if file.endswith('created.csv'):
                resultCreated.loc[regionName] = {'created': data.iloc[ts//10000 -1,-1]}
            if file.endswith('deleted.csv'):
                resultDeleted.loc[regionName] = {'deleted': data.iloc[ts//10000 -1,-1]}
    return (resultCreated, resultDeleted)

def rankRegions(sim, critera = 'created'):
    # this just ranks regions based on magnitude of creation or deletion events
    # critera can be 'created' or 'deleted'
    resultCreated, resultDeleted = getAvgRegion(0, 990000, sim)
    if critera == 'created':
        return resultCreated.sort_values(by='avg', ascending=False).index
    else:
        return resultDeleted.sort_values(by='avg', ascending=False).index
    
def compareDisable(disableTS):
    # find large increases in deleted events
    createdBefore, deletedBefore = getAvgRegion(0, disableTS, sim='disable')
    createdAfter, deletedAfter = getAvgRegion(disableTS, 990000, sim='disable')
    created = pd.concat([createdBefore, createdAfter], axis=1)
    created.columns = ['before', 'after']
    created['change'] = created['after'] - created['before']
    created = created.sort_values(by='change', ascending=False)
    deleted = pd.concat([deletedBefore, deletedAfter], axis=1)
    deleted.columns = ['before', 'after']
    deleted['change'] = deleted['after'] - deleted['before']
    deleted = deleted.sort_values(by='change', ascending=False)
    return (created, deleted)

def rankDisable():
    # returns area list sorted based on most deletions
    created, deleted = compareDisable(disableTS=100000)
    return deleted.index

def compareStimulus(chunk = False, useBaseline = True, returnRankings = False):
    # find large increases in creations
    # stimTS = [150000, 152000,200000,202000,250000,252000, 300000,302000, 350000,352000,400000,402000, 
    #           450000,452000,500000,502000,550000,552000,652000,654000, 702000,704000,]
    stimTS = [150000, 200000,250000, 300000, 350000,400000, 450000,500000,550000,652000, 702000,]
    stimTS = [0] + stimTS + [990000]
    created = pd.DataFrame({'area': regionList}).set_index('area', inplace=True)
    deleted = pd.DataFrame({'area': regionList}).set_index('area', inplace=True)
    for i in range(1, len(stimTS)):
        if chunk:
            createdPeriod, deletedPeriod = getAvgRegion(stimTS[i-1], stimTS[i], sim='stimulus')
        else:
            createdPeriod, deletedPeriod = getTimestepRegion(stimTS[i], sim='stimulus')
        colName = i
        createdPeriod.columns = [colName]
        deletedPeriod.columns = [colName]
        created = pd.concat([created, createdPeriod], axis=1)
        deleted = pd.concat([deleted, deletedPeriod], axis=1)

        dfLen = len(created.columns)
        for iCol in range(dfLen):
            col = created.columns[iCol]
            if iCol == dfLen - 1:
                pass # this means it's the first column aka the baseline
            if useBaseline:
                created[col] = created[col] - created[created.columns[0]]
                deleted[col] = deleted[col] - deleted[deleted.columns[0]]
            else: # iterate from the back forward to avoid double subtration
                created[created.columns[dfLen - iCol - 1]] = created[created.columns[dfLen - iCol - 1]] - created[created.columns[dfLen - iCol - 2]]
                deleted[deleted.columns[dfLen - iCol - 1]] = deleted[deleted.columns[dfLen - iCol - 1]] - deleted[deleted.columns[dfLen - iCol - 2]]

        if returnRankings:
            # change absolute values to rankings
            for col in created.columns:
                created[col] = created[col].rank()
                deleted[col] = deleted[col].rank()
    return (created, deleted)

created, deleted = compareStimulus()

In [44]:
# after 1| 1-2 | 2-3 |...| before the end
created

,1,2,3,4,5,6,7,8,9,10,11,12
area,,,,,,,,,,,,
area_43,0,612,768,875,908,970,762,1008,670,0,0,0
area_30,0,1392,1594,1959,2100,2102,1884,2072,1720,0,0,0
area_9,0,676,812,930,894,1014,860,1124,790,0,0,0
area_5,0,1914,2230,2540,2198,2282,2264,2920,2072,0,0,0
area_41,0,1088,1220,1406,1312,1436,1264,1654,1274,0,0,0
area_46,0,740,996,1052,1244,1304,970,1306,980,0,0,0
area_35,0,612,698,1068,1318,1272,1112,1332,1020,0,0,0
area_34,0,980,1360,1572,2288,2186,1834,1808,1738,0,0,0
area_45,0,1098,1366,1636,1348,1634,1330,1634,1462,0,0,0


In [58]:
chunks=[False,True]
useBaselines=[False,True]
returnRankings=[False,True]
result = {}
for chunk in chunks:
    for useBaseline in useBaselines:
        for returnRanking in returnRankings:
            created, deleted = compareStimulus(chunk, useBaseline,returnRanking)
            created = json.loads(created.to_json())
            deleted = json.loads(deleted.to_json())
            result[f'{+chunk}{+useBaseline}{+returnRanking}'] = [created,deleted]
with open(str(CURRENT_DIRECTORY)+"/SciVisContest23/stimulus_rank.json", "w") as outfile:
        json.dump(result, outfile)
        

In [59]:
created

{'1': {'area_43': 24.5,
  'area_30': 24.5,
  'area_9': 24.5,
  'area_5': 24.5,
  'area_41': 24.5,
  'area_46': 24.5,
  'area_35': 24.5,
  'area_34': 24.5,
  'area_45': 24.5,
  'area_8': 24.5,
  'area_14': 24.5,
  'area_4': 24.5,
  'area_1': 24.5,
  'area_32': 24.5,
  'area_24': 24.5,
  'area_22': 24.5,
  'area_6': 24.5,
  'area_12': 24.5,
  'area_42': 24.5,
  'area_47': 24.5,
  'area_21': 24.5,
  'area_25': 24.5,
  'area_38': 24.5,
  'area_20': 24.5,
  'area_26': 24.5,
  'area_39': 24.5,
  'area_0': 24.5,
  'area_16': 24.5,
  'area_23': 24.5,
  'area_33': 24.5,
  'area_28': 24.5,
  'area_27': 24.5,
  'area_31': 24.5,
  'area_13': 24.5,
  'area_15': 24.5,
  'area_2': 24.5,
  'area_18': 24.5,
  'area_11': 24.5,
  'area_19': 24.5,
  'area_29': 24.5,
  'area_44': 24.5,
  'area_40': 24.5,
  'area_36': 24.5,
  'area_7': 24.5,
  'area_17': 24.5,
  'area_37': 24.5,
  'area_10': 24.5,
  'area_3': 24.5},
 '2': {'area_43': 21.0,
  'area_30': 44.0,
  'area_9': 27.0,
  'area_5': 48.0,
  'area_41': 

In [ ]:
rankRegions()

In [37]:
rankRegions('stimulus').tolist()

['area_8',
 'area_5',
 'area_6',
 'area_30',
 'area_22',
 'area_34',
 'area_20',
 'area_45',
 'area_31',
 'area_41',
 'area_39',
 'area_37',
 'area_24',
 'area_11',
 'area_1',
 'area_4',
 'area_46',
 'area_3',
 'area_35',
 'area_2',
 'area_42',
 'area_14',
 'area_26',
 'area_9',
 'area_21',
 'area_47',
 'area_43',
 'area_16',
 'area_0',
 'area_40',
 'area_32',
 'area_10',
 'area_38',
 'area_17',
 'area_29',
 'area_13',
 'area_28',
 'area_27',
 'area_7',
 'area_33',
 'area_15',
 'area_19',
 'area_12',
 'area_44',
 'area_36',
 'area_18',
 'area_23',
 'area_25']

In [38]:
sims = ['calcium', 'stimulus', 'no-network', 'disable']
connectType = ["created","deleted"]
connectRank={}
for connect in connectType:
    connectRank[connect] = {}
    for sim in sims:
        connectRank[connect][sim] = rankRegions(sim,connect).tolist()

In [40]:
import json
with open(str(CURRENT_DIRECTORY)+"/regionRanking.json", "w") as outfile:
    json.dump(connectRank, outfile)